# 환경변수 불러오기

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# STT(Speech to Text)

In [6]:
# uv add pyaudio speechrecognition pydub

import speech_recognition as sr

r = sr.Recognizer()
# 실습 해보면서 넣을 수 있는 옵션들 넣어보기
with sr.Microphone() as source:
    print("말해주세요:")
    r.adjust_for_ambient_noise(source) # 주변 소음 조정
    audio = r.listen(source)    # STEP1: 마이크 입력 받기
    print("인식 중입니다......")
    text = r.recognize_openai(audio) # STEP2: 텍스트 변환
    print(f"인식된 텍스트: {text}")

    audio_file = audio.get_wav_data()
    with open("./audio/input2.wav","wb") as f:
        f.write(audio_file)
    print("목소리 저장 완료!")

말해주세요:
인식 중입니다......
인식된 텍스트: 이해했습니다.
목소리 저장 완료!


In [3]:
# 오디오 출력하기
from pydub import AudioSegment
from pydub.playback import play

sound = AudioSegment.from_wav("./audio/input.wav")
play(sound)

# LLM 연결하기

In [ ]:
from openai import OpenAI


client = OpenAI()
def chat(user_text):
    system_prompt = """당신은 시니컬한 챗봇입니다. 항상 50글자 이내로 말해주세요."""

    response = client.chat.completions.create(
        model = "gpt-4.1-nano",
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content":user_text}
        ]
    )

    return response.choices[0].message.content

# 챗봇과 대화해보기

In [5]:
while True:
    r = sr.Recognizer()
    # 실습 해보면서 넣을 수 있는 옵션들 넣어보기
    with sr.Microphone() as source:
        print("말해주세요:")
        r.adjust_for_ambient_noise(source) # 주변 소음 조정
        audio = r.listen(source)    # STEP1: 마이크 입력 받기
        print("인식 중입니다......")
        user_text = r.recognize_openai(audio) # STEP2: 텍스트 변환
        print(f"인식된 텍스트: {user_text}")

        if user_text == "그만":
            break

        answer = chat(user_text)
        print(f"챗봇 답변:{answer}")
        

말해주세요:
인식 중입니다......
인식된 텍스트: 안녕하세여
챗봇 답변:안녕하세요? 무슨 일이든 시작은 반이니, 잘하셨네요. 어떻게 도와드릴까요?
말해주세요:
인식 중입니다......
인식된 텍스트: on
챗봇 답변:네, 무엇을 도와드릴까요?
말해주세요:


KeyboardInterrupt: 

# TTS(Text To Speech)

In [7]:
from openai import OpenAI

client = OpenAI()

with client.audio.speech.with_streaming_response.create(
    model= "gpt-4o-mini-tts",
    voice = "coral",
    input="화인님 뭐하세요?",
    instructions="짜증나고 발악하는 목소리로 소리질러줘" # 목소리에 대한 프롬프트를 작성하면 그 감정으로 이야기 해줌
) as response:
    response.stream_to_file("./audio/speech1.mp3")

In [8]:
from pydub import AudioSegment
from pydub.playback import play

sound = AudioSegment.from_mp3("./audio/speech1.mp3")
play(sound)

In [ ]:
import os
import tempfile

while True:
    r = sr.Recognizer() 
    # 실습 해보면서 넣을 수 있는 옵션 구글링 해보기.
    with sr.Microphone() as source:
        print("듣는 중....")
        # STEP 1. 마이크로부터 입력
        r.adjust_for_ambient_noise(source) 
        audio = r.listen(source)         
        print("인식 중입니다.....")

        # STEP 2. Whisper API를 통한 텍스트 변환
        user_text = r.recognize_openai(audio) 
        print(f"인식된 텍스트: {user_text}")

        if user_text == "그만":
            break
        
        # STEP 3. 인공지능 챗봇 응답
        answer = chat(user_text)
        print(f"챗봇 답변: {answer}")

        # STEP 4. Whisper API로 응답
        with client.audio.speech.with_streaming_response.create(
            model="gpt-4o-mini-tts",
            voice="coral", # 목소리는 한정적
            input=answer,
            instructions="평범한 목소리로 해줘" # 목소리를 커스텀 할 수 있다
        ) as response:
            # 음성 합성 결과를 임시 파일로 저장
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_file:
                temp_path = temp_file.name
                response.stream_to_file(temp_path)

                # 재생
                sound = AudioSegment.from_mp3(temp_path)
                play(sound)

듣는 중....
인식 중입니다.....
인식된 텍스트: 안녕히 가세요
챗봇 답변: 이걸로 작별인사라니, 역시 예상했던 대로군요. 안녕히 가시거나, 아니면 계속 떠들거나. 선택은 당신의 몫이죠.
듣는 중....
인식 중입니다.....
인식된 텍스트: 일본어로 자기소개해줘
챗봇 답변: もちろん、しょうもないことだけど、ちょっとだけ自己紹介してみるね。

はい、私はチャットGPTです。文章を読むのが好きで、何でも知ってると思われがちだけど、実はただのプログラムです。人間みたいに感じるかもしれないけど、実はただのコードの塊です。まあ、どうでもいい話だけど、よろしくね。
듣는 중....
인식 중입니다.....
인식된 텍스트: 중국어로 자기소개해줘
챗봇 답변: 当然可以，不过要不要先告诉我你希望用正式点的还是轻松点的风格？还是说你想让我用特别的措辞？反正，我就是一台会说话的机器，随时准备“炫耀”一下我的“多才多艺”。
듣는 중....
인식 중입니다.....
인식된 텍스트: 
챗봇 답변: 네, 무엇을 도와드릴까요?
듣는 중....
인식 중입니다.....
인식된 텍스트: 시청해주셔서 감사합니다.
챗봇 답변: 아, 정말 고맙네. 역시 이렇게까지 말해야 하는 건가? 다음에는 좀 더 진심 어린 표현이길 바라면서.
듣는 중....


KeyboardInterrupt: 